In [2]:
import pandas as pd
import numpy as np

In [ ]:
# leitura e preparação
df = pd.read_csv("dados_inmet.csv", sep=";", decimal=",", encoding="latin1")
df["Data"] = pd.to_datetime(df["Data"], dayfirst=True)

# nomes das colunas (ajuste se necessário)
COL_RAD = "RADIACAO GLOBAL (Kj/m²)"
COL_TEMP = "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)"
COL_UMID = "UMIDADE RELATIVA DO AR, HORARIA (%)"
COL_PREC = "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"

# garantir numéricos
for col in [COL_RAD, COL_TEMP, COL_UMID, COL_PREC]:
    df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)

# agregação diária
daily = df.groupby(df["Data"].dt.date).agg(
    total_rad=(COL_RAD, "sum"),
    max_temp=(COL_TEMP, "max"),
    min_temp=(COL_TEMP, "min"),
    mean_umid=(COL_UMID, "mean"),
    total_prec=(COL_PREC, "sum")
)

# derivar amplitude térmica
daily["amp_temp"] = daily["max_temp"] - daily["min_temp"]

# quantis para thresholds autoajustados
q_rad_low  = daily["total_rad"].quantile(0.33)
q_rad_high = daily["total_rad"].quantile(0.66)

def classificar(row):
    rad = row["total_rad"]
    amp = row["amp_temp"]
    umi = row["mean_umid"]
    prec = row["total_prec"]

    # Dia ensolarado
    if (rad >= q_rad_high) and (amp >= 8) and (umi <= 70) and (prec == 0):
        return "ensolarado"

    # Dia nublado
    if (rad <= q_rad_low) and (amp <= 5) and (umi >= 80):
        return "nublado"

    # Tudo que sobra fica como parcialmente nublado
    return "parcialmente nublado"

daily["classificacao"] = daily.apply(classificar, axis=1)

resultado = daily[["classificacao"]].reset_index().rename(columns={"Data":"dia"})
print(resultado)

# (opcional) salvar em CSV
# resultado.to_csv("dias_classificados.csv", index=False)